In [1]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
from scipy.interpolate import Rbf
output_notebook()

Loading BokehJS ...

In [2]:
from pydgrid.pydgrid import grid

In [3]:
sys1 = grid()
sys1.read('./examples/cigre/cigre_europe_residential.json')  # Load data
sys1.pf()  
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

V_an  = np.array([item['v_an']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
V_bn  = np.array([item['v_bn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
V_cn  = np.array([item['v_cn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
Pos_x = np.array([item['pos_x'] for item in sys1.buses])
Pos_y = np.array([item['pos_y'] for item in sys1.buses])
x_min = np.min(Pos_x)
x_max = np.max(Pos_x)
y_min = np.min(Pos_y)
y_max = np.max(Pos_y)

x_margin = 150
y_margin = 100

x_min2 = x_min-x_margin
x_max2 = x_max+x_margin
y_min2 = y_min-y_margin
y_max2 = y_max+y_margin

Pos_x_e = np.hstack((x_min2, x_min2, Pos_x, x_max2, x_max2))
Pos_y_e = np.hstack((y_min2, y_max2, Pos_y, y_max2, y_min2))
V_an_e  = np.hstack((1,1, V_an, 1,1))
V_bn_e  = np.hstack((1,1, V_bn, 1,1))
V_cn_e  = np.hstack((1,1, V_cn, 1,1))
rbfi_a = Rbf(Pos_x_e, Pos_y_e, V_an_e, function='linear')
rbfi_b = Rbf(Pos_x_e, Pos_y_e, V_bn_e)
rbfi_c = Rbf(Pos_x_e, Pos_y_e, V_cn_e)

x =  np.linspace(x_min2,x_max2, 500)
y =  np.linspace(y_min2,y_max2, 500)
X, Y = np.meshgrid(x, y)
Z_a = rbfi_a(X, Y)
Z_b = rbfi_b(X, Y)
Z_c = rbfi_c(X, Y)

## CIGRE LV European System

### Graph with obtained results

In [10]:
sys1.s_radio_scale = 0.1
sys1.bokeh_tools()

p = figure(width=600, height=800,
           title='European LV Network (CIGRE)', x_range=[-150,150], y_range=[-350, 50])

p.image(image=[Z_a], x=x_min2, y=y_min2, dw=(x_max2-x_min2), dh=(y_max2-y_min2), palette="Spectral11")


# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=1, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size='s_radio', color='s_color', alpha=1)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

### Interaction with powers

In [24]:
sys1.read('./examples/cigre/cigre_europe_residential.json')  # Load data
sys1.pf()  
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

s_0_3pn = np.copy(sys1.pq_3pn) # 
s_0_1p  = np.copy(sys1.pq_1p)  # 
sys1.bokeh_tools()
p = figure(width=800, height=400,
           title='Voltage vs load powers', 
           x_range = [40,-350], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-400,300],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-400,300],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_loads(load_factor=1.0):

    sys1.pq_1p = load_factor*s_0_1p
    sys1.pq_3pn = np.copy(load_factor*s_0_3pn)
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()
    v_an_m = np.abs(sys1.V_node[sys1.node_1_sorter])
    v_bn_m = np.abs(sys1.V_node[sys1.node_2_sorter])
    v_cn_m = np.abs(sys1.V_node[sys1.node_3_sorter])
    sys1.bus_data['v_an'] = v_an_m
    sys1.bus_data['v_bn'] = v_bn_m
    sys1.bus_data['v_cn'] = v_cn_m
    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [25]:
from ipywidgets import interact
interact(update_loads, load_factor=(-1,1.2, 0.1))

A Jupyter Widget

<function __main__.update_loads>

In [19]:
sys1.I_node[5]

array([-100.05476146-276.73605679j])

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame()

In [22]:
df['nodes'] = sys1.nodes
df['I_node_m'] = np.abs(sys1.I_node)
df['V_node_m'] = np.abs(sys1.V_node)
df['phi'] = np.angle(sys1.I_node, deg=True) - np.angle(sys1.V_node, deg=True)
df['s'] = np.conjugate(sys1.I_node)*sys1.V_node
df['p'] = df['s'].real
df['q'] = df['s'].imag

In [23]:
df

,nodes,I_node_m,V_node_m,phi,s,p,q
0,R0.1,12.279686,11547.000000,-20.016659,(133228.23479+48534.9835399j),133228.234790,48534.983540
1,R0.2,12.194776,11547.000000,-20.114054,(132224.878309+48424.2132824j),132224.878309,48424.213282
2,R0.3,12.219433,11547.000000,-19.720389,(132822.486406+47610.6662094j),132822.486406,47610.666209
3,R1.1,294.305597,226.183736,161.716058,(-63206.3937442-20883.865419j),-63206.393744,-20883.865419
4,R1.2,294.318023,226.377454,161.923802,(-63338.5714189-20673.1172036j),-63338.571419,-20673.117204
5,R1.3,294.268246,227.024469,-198.224433,(-63455.0167918-20892.9366499j),-63455.016792,-20892.936650
6,R1.4,0.129778,0.488038,-253.446536,(-0.0180451863612-0.0607114369727j),-0.018045,-0.060711
7,R11.1,22.619924,221.082177,161.816080,(-4751.11743079-1560.61051413j),-4751.117431,-1560.610514
8,R11.2,22.542696,221.818706,161.791130,(-4749.99046844-1562.53229662j),-4749.990468,-1562.532297
9,R11.3,22.431808,222.842141,-198.191793,(-4748.89837058-1560.60457991j),-4748.898371,-1560.604580
